In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
df = pd.read_csv("College.csv", index_col=0)

In [26]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [27]:
df.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [28]:
# Map Private: "Yes"/"No" -> 1/0
df["Private"] = df["Private"].astype(str).str.strip().str.lower().map({"yes":1, "no":0})

y = df["Apps"].to_numpy()
X = df.drop(columns=["Apps"]).to_numpy()  # all other columns as predictors

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [29]:
def report_model(name, y_true, y_pred, extra: dict = None):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    out = {"model": name, "test_MSE": mse, "test_RMSE": rmse}
    if extra:
        out.update(extra)
    return out

results = []

In [30]:
#b
ols = LinearRegression()
ols.fit(X_train, y_train)
y_pred_ols = ols.predict(X_test)
results.append(report_model("OLS", y_test, y_pred_ols))


/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [31]:
#c
alphas = np.logspace(-3, 6, 100)  # 1e-3 ... 1e6
ridge_cv = RidgeCV(alphas=alphas, scoring="neg_mean_squared_error", cv=10)

ridge_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("ridge", ridge_cv)
])
ridge_pipe.fit(X_train, y_train)

ridge_best_alpha = ridge_pipe.named_steps["ridge"].alpha_
y_pred_ridge = ridge_pipe.predict(X_test)
results.append(report_model("Ridge", y_test, y_pred_ridge,
                            extra={"best_lambda": ridge_best_alpha}))


/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ 

In [32]:
#d
lasso_cv = LassoCV(cv=10, random_state=42, max_iter=20000)
lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("lasso", lasso_cv)
])
lasso_pipe.fit(X_train, y_train)

lasso_best_alpha = lasso_pipe.named_steps["lasso"].alpha_
y_pred_lasso = lasso_pipe.predict(X_test)

# number of non-zero coefficients (excluding intercept; pipeline has no explicit intercept)
nonzero = np.count_nonzero(lasso_pipe.named_steps["lasso"].coef_)

results.append(report_model("Lasso", y_test, y_pred_lasso,
                            extra={"best_lambda": lasso_best_alpha,
                                   "n_nonzero_coefs": int(nonzero)}))


/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/quant1wen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [33]:
summary = pd.DataFrame(results)
print(summary)

   model      test_MSE    test_RMSE  best_lambda  n_nonzero_coefs
0    OLS  1.492443e+06  1221.655999          NaN              NaN
1  Ridge  1.492427e+06  1221.649493     0.001000              NaN
2  Lasso  1.478956e+06  1216.123415     3.721118             17.0
